## Sampling Average Surface Brightness within Effective Radii

### October 2022

Using surface brightness profile results.

In [67]:
# math, array manipulation, etc.
import numpy as np

import os
import sys

# timing
from timeit import default_timer
from tqdm import tqdm

import pandas as pd

import astropy.io.fits as fits
from astropy.table import Table                    # Table data structure
import astropy.units as u

# necessary utilities from scipy, astropy and photutils
from scipy.optimize import differential_evolution
from scipy.ndimage import maximum_filter, gaussian_filter
from astropy.modeling import functional_models
from astropy.convolution import convolve, Gaussian2DKernel
from astropy.stats import gaussian_fwhm_to_sigma, SigmaClip
from photutils.background import Background2D, StdBackgroundRMS
from photutils.segmentation import deblend_sources, SegmentationImage, detect_sources

from scipy.stats import gaussian_kde

# plots
import matplotlib.pyplot as plt

#%matplotlib inline

# astronomical image fitting software Imfit, wrapped in Python
# https://github.com/perwin/pyimfit
# https://pyimfit.readthedocs.io/en/latest/
import pyimfit

# SEP
import sep

project_path = '/Users/aidan/Desktop/sl_project/'
sys.path.append(project_path + 'galaxy-galaxy-sel/')

from lensphoto import LensPhoto, fwhm2sigma

fwhm_r = 0.95
fwhm_g = 1.11

std_psf_r = fwhm2sigma(fwhm_r)
std_psf_g = fwhm2sigma(fwhm_g)

from aidan_utils import plot_prettier

plot_prettier()


In [8]:
results_path = project_path + 'production/'

# read lens results as pandas dataframe
lens_results = pd.read_csv(results_path + 'lens_results.csv')

rband_imgs = fits.open(results_path + 'lens_images_r.fits')
gband_imgs = fits.open(results_path + 'lens_images_g.fits')


In [151]:
# define params that we need
params = ['halflightrad_r', 'sersicindex_r'
          , 'halflightrad_g', 'sersicindex_g'
          , 'ellipticity', 'positionangle']

index = 0

img_r = rband_imgs[index].data
img_g = gband_imgs[index].data

photo = LensPhoto(img_r = img_r, img_g = img_g
                  , std_psf_r = std_psf_r, std_psf_g = std_psf_g
                  , desid = '0'
                 )


/Users/aidan/Desktop/sl_project/galaxy-galaxy-sel/lensphoto.py:215: RuntimeWarning: invalid value encountered in sqrt
  std_gal = np.sqrt(num * img_gal)
/Users/aidan/Desktop/sl_project/galaxy-galaxy-sel/lensphoto.py:215: RuntimeWarning: invalid value encountered in sqrt
  std_gal = np.sqrt(num * img_gal)


  0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

In [152]:
means   = np.zeros(len(params))
spreads = np.zeros(len(params))

for i, p in enumerate(params):
    
    p16 = lens_results[p + '_16'][index]
    p50 = lens_results[p + '_50'][index]
    p84 = lens_results[p + '_84'][index]
    
    minus1 = p50 - p16
    plus1  = p84 - p50
    print(p50)
    print(minus1, plus1)
    
    s = np.sqrt(minus1**2 + plus1**2)
    
    means[i] = p50
    spreads[i] = s


4.692764991554007
0.3116583889155349 0.36654910392975637
1.3764772122065112
0.13824761791974205 0.12155884822778695
6.093812238964468
0.5768489268290473 0.5751657403987824
0.7576356480886506
0.1353515201587363 0.17003387851714857
0.2743242572275551
0.04321049340888919 0.04832227701998992
2.996398494261796
2.0205743293490985 2.875441571199574


In [158]:
def generate_samples(ind):
    
    mean = means[ind]
    spread = spreads[ind]
    
    sample = np.random.normal(loc=mean, scale=spread, size=1000)
    
    if np.min(sample) < 0:
        sample = np.abs(sample)
    
    return sample

rrsample = generate_samples(0)
nrsample = generate_samples(1)
rgsample = generate_samples(2)
ngsample = generate_samples(3)
esample = generate_samples(4)
thsample = generate_samples(5)

# I need to find a way to use my magnitudes (converted into flux) to extract a surface brightness I_e

# maybe perform the exact same flux measurement (kron_rad * 15) and divide mag2flux value by that flux;
# that should give surface brightness I_e at R_e

# then I can perform the flux measurement through circular aperture R_e, divide by solid angle within that
# circle, and that's <I_e>!
